## Extending input sequences for CRISPR base editing modeling
Caleb Lareau

The way this works is that we'll import the old .fasta files, pull an extended sequence, and then export new .fasta files. The reference genome is hg38. For super long sequences, we may run into more edge cases where the base that it is being modeled runs off of the transcript and into gDNA, which doesn't necessarily represent reality on the mRNA molecule. 

In [ ]:
# Run this once to get the dependencies and then delete it
install.packages("data.table")
install.packages("seqinr")
install.packages("stringr")
install.packages("BiocManager")
BiocManager::install("BSgenome.Hsapiens.UCSC.hg38")
BiocManager::install("Biostrings")
BiocManager::install("GenomicRanges")

In [1]:
library(BSgenome.Hsapiens.UCSC.hg38)
library(Biostrings)
library(data.table)
library(seqinr)
library(stringr)
library(GenomicRanges)

# Crude pass that pulls from gDNA -- does not consider any potential splicing
getSequenceSimple <- function(gr_query, reverse_complement = FALSE){
  
  gs <- getSeq(BSgenome.Hsapiens.UCSC.hg38, gr_query)
  
  # Flip the sequence if we want to
  if(reverse_complement){
    gs_rc <- reverseComplement(gs)
    gs <- gs_rc
  }
  
  return(as.character(gs))
}




Loading required package: BSgenome
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors
Loading required package: stats4

Attaching

In [2]:

# Function that takes a current fasta file and then pads
# the desired amount, resulting in a new fasta file
# with sequence with 2*new_pad + 1
repad_fasta <- function(path_to_fasta, new_pad = 100){
    dt <- fread(path_to_fasta, header = FALSE)
    headers <- gsub(">", "", dt[rep(c(TRUE,FALSE),dim(dt)[1]/2),][[1]])
    sequences <- dt[rep(c(FALSE,FALSE),dim(dt)[1]/2),][[1]]

    # Munge the headers a bit
    split_mat <- str_split_fixed(headers, "_", 5)
    genome_positions <- str_split_fixed(split_mat[,2], "-", 2)

    granges_seqs <- makeGRangesFromDataFrame(
        data.frame(
        chr = genome_positions[,1],
        start = as.numeric(genome_positions[,2]) - new_pad,
        end = as.numeric(genome_positions[,2]) + new_pad
        )
    )
    boo_forward <- split_mat[,3] == "F"
    new_seqs_F <- getSequenceSimple(granges_seqs[boo_forward])
    new_seqs_R <- getSequenceSimple(granges_seqs[!boo_forward], reverse_complement = TRUE)

    # Export the sequences to a new fasta file
    fasta_names <- c(headers[boo_forward], headers[!boo_forward])
    new_fasta_file <- gsub(".fasta.gz", paste0("_pad",as.character(new_pad),".fasta"), path_to_fasta)
    write.fasta(as.list(c(new_seqs_F, new_seqs_R)), fasta_names, new_fasta_file, open = "w")
    system(paste0("gzip ", new_fasta_file))

}

In [10]:
# Here's a minimal function call but this is best done in a loop most likely. 
# One can use the list.files() command in R to show files in a path 
# And then loop over them. 

for (x in list.files("../data/processed/ABE/ABE-sequence/")) {
    repad_fasta(paste("../data/processed/ABE/ABE-sequence/", x, sep = ""), 500)
}